# Python Environment Setup

In [80]:
!pip3 install virtualenv

In [81]:
!python3 -m venv myenv

In [82]:
!source myenv/bin/activate

# This code will create a spreadsheet for all input, correct output and system output

In [92]:
import json
import pandas as pd

## creating the necessary columns as required

In [160]:
column_names = ["query_type","relation", "query","answer","gold facts","system facts", "original output","system output"]
my_data = pd.DataFrame(columns = column_names)

In [161]:
query_type = []
relation = []
query = []
correct = []
gold_op = []
ssg_op = []
gold_facts = []
system_facts = []


## This method will get query, answer, relation and query type from test file.

In [162]:
def get_data_from_original(path):
    with open(path) as json_file:
        for line in json_file:
            db = json.loads(line)
            for all_query in db["queries"]:
                query.append(all_query['query'])
                correct.append(all_query['answer'])
                relation.append(all_query['relation'])
                gold_facts.append(all_query["derivations"])
                if "arg" in all_query['type'] or "min" in all_query['type'] or "max" in all_query['type']:
                    query_type.append("min/max")
                else:
                    query_type.append(all_query['type'])



In [163]:
get_data_from_original("mtab_onlytable/balanced_test.jsonl")

## This function will get the gold facts and predicted facts from the predicted file

In [164]:
def get_data_from_prediction(path):
    with open(path) as json_file:
        db = json.load(json_file)

        for idx,data in enumerate(db):
            temp_ssg_op = []
            temp_system_facts = []
            gold_op.append(data['gold_facts'])
            for row in data["ssg_output"]:
                for i in row:
                    temp_ssg_op.append(i[0])
                    temp_system_facts.append(i[1])
            ssg_op.append(temp_ssg_op)
            system_facts.append(temp_system_facts)
                    
            
            

In [165]:
get_data_from_prediction("mtab_onlytable/balanced_test_0.7_ssg_sup.json")

In [166]:
len(system_facts)

86

In [167]:
my_data["query_type"] = query_type
my_data['query'] = query
my_data['relation'] = relation
my_data["answer"] = correct
my_data["gold facts"] = gold_facts
my_data["system facts"] = system_facts
my_data["original output"] = gold_op
my_data["system output"] = ssg_op

## We can analyze the correct output and system output to see whether their is an exact match or some partial match.

In [168]:
my_data.head()

,query_type,relation,query,answer,gold facts,system facts,original output,system output
0,min/max,P17,Which league has the fewest number of particip...,"[Sánchez, Bobsleigh at the 2002 Winter Olympic...","[Sánchez [SEP] States, Bobsleigh at the 2002 W...","[Sánchez country States, Bobsleigh at the 2002...","[[0], [2], [4], [7]]","[0, 2, 4, 7]"
1,bool,P571,Is 2011 Mumbai's inception date?,[TRUE],[TRUE],[Mumbai established 2011],[[3]],[3]
2,count,P17,How many participating country does Sánchez h...,[2],"[States, Philippines]","[Sánchez country States, Bobsleigh at the 2002...","[[0], [4]]","[0, 2, 4]"
3,bool,P17,Is Nepal a participating country of Wuhan?,[TRUE],[TRUE],"[Wuhan country Nepal, Bobsleigh at the 2002 Wi...",[[0]],"[0, 8]"
4,bool,P571,Is Mumbai existed in 2007?,[TRUE],[TRUE],"[Mumbai established 2011, Mumbai established 2...",[[9]],"[5, 7, 9]"


In [169]:
len(my_data)

86

## Save the dataframe as spreadsheet

In [170]:
my_data.to_csv('mtab_onlytable/mtab_onlytable_sheet_test.tsv',sep='\t', index=False)

